데이터 수집 및 전처리 코드는 파일이 큰 관계로 시험용으로 구성된 전체 코드만 제공됩니다.

# 기초 데이터 수집

## TMDB 데이터 수집

In [1]:
import file_collector.tmdb_collector as tdc
import time
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")

In [4]:
start_date = '2025-11-01'
end_date = '2025-11-10'

OUTPUT_DIR = "files"

### 영화

In [10]:
if __name__ == "__main__":
    total_start = time.time()

    # 기간을 3개월 단위로 분할
    periods = tdc.generate_date_periods(start_date, end_date, months=3)

    print(f"\n전체 기간: {start_date} ~ {end_date}")
    print(f"분할 기간: {len(periods)}개")
    print("="*90)

    # ID 데이터를 집합으로 변환하여 중복 제거
    all_ids = set()

    for idx, (period_start, period_end) in enumerate(periods, 1):
        print(f"\n[{idx}/{len(periods)}] 기간: {period_start} ~ {period_end}")

        # 기간내 영화 ID 수집
        current_ids_set = tdc.collect_movie_ids(period_start, period_end)
        # ID를 집합에 추가
        all_ids.update(current_ids_set)

        print(f"해당 기간 내 수집된 ID: {len(current_ids_set):,}개 (누적 ID: {len(all_ids):,}개)")

    # 집합을 리스트로 변환
    id_list = list(all_ids)

    print(f"\n{'='*90}")
    print(f"최종 ID 개수: {len(id_list):,}개")

    print(f"\n{'='*90}")
    print("상세 정보 멀티스레드 수집 시작...\n")

    results = []
    # 3. 상세 정보 수집 단계: ID 리스트를 사용
    with ThreadPoolExecutor(max_workers=30) as executor:
        # 딕셔너리 대신 ID 자체를 future에 매핑
        futures = {executor.submit(tdc.fetch_movie_details, movie_ids): movie_ids for movie_ids in id_list}

        for future in tqdm(as_completed(futures), total=len(futures), desc="영화 상세 수집"):
            detail = future.result()
            if detail:
                results.append(detail)

    print(f"\n상세 정보 수집 완료: {len(results):,}개")

    # 저장
    df = pd.DataFrame(results)
    # 컬럼 이름이 'first_air_date'라고 가정
    df = df.sort_values(["release_date", "popularity"], ascending=[True, False])
    df.to_csv(f"{OUTPUT_DIR}/movies_test.csv", index=False, encoding="utf-8-sig")

    elapsed = time.time() - total_start

    print("\n" + "="*90)
    print("========================== DONE ==========================")
    print(f"총 데이터: {len(df):,}개")
    print(f"소요시간: {elapsed/60:.2f}분 ({elapsed:.2f}초)")
    print(f"저장 파일: movies.csv")
    print("="*90)


전체 기간: 2025-11-01 ~ 2025-11-10
분할 기간: 1개

[1/1] 기간: 2025-11-01 ~ 2025-11-10

📺 영화 ID 수집: 2025-11-01 ~ 2025-11-10

[기간 1/1] 2025-11-01 ~ 2025-11-10
  [영화] 총 1,513개 (76페이지) → 수집: 76페이지


  누적 ID: 1,513개

최종 수집: 1,513개 영화 ID

해당 기간 내 수집된 ID: 1,513개 (누적 ID: 1,513개)

최종 ID 개수: 1,513개

상세 정보 멀티스레드 수집 시작...



영화 상세 수집: 100%|██████████| 1513/1513 [00:26<00:00, 57.78it/s]



상세 정보 수집 완료: 1,513개

========================== DONE ==========================
총 데이터: 1,513개
소요시간: 0.48분 (28.86초)
저장 파일: movies.csv


### 드라마

In [11]:
if __name__ == "__main__":
    total_start = time.time()

    # 기간을 N개월 단위로 분할
    periods = tdc.generate_date_periods(start_date, end_date, months=12)

    print(f"\n전체 기간: {start_date} ~ {end_date}")
    print(f"분할 기간: {len(periods)}개")
    print("=" * 90)

    # ID 데이터를 집합으로 변환하여 중복 제거
    all_ids = set()

    for idx, (period_start, period_end) in enumerate(periods, 1):
        print(f"\n[{idx}/{len(periods)}] 기간: {period_start} ~ {period_end}")

        # 기간내 TV 시리즈 ID 수집
        current_ids_set = tdc.collect_tv_ids(period_start, period_end)
        all_ids.update(current_ids_set)

        print(f"해당 기간 내 수집된 ID: {len(current_ids_set):,}개 (누적 ID: {len(all_ids):,}개)")

    # 집합을 리스트로 변환
    id_list = sorted(list(all_ids))

    print(f"\n{'=' * 90}")
    print(f"최종 ID 개수: {len(id_list):,}개")

    # ============================================================
    # 2단계: TV 시리즈 + 시즌 상세 정보 수집 (멀티스레드)
    # ============================================================
    print(f"\n{'=' * 90}")
    print("상세 정보 멀티스레드 수집 시작...\n")

    series_results = []
    seasons_results = []

    with ThreadPoolExecutor(max_workers=20) as executor:
        # 시리즈 + 시즌 동시 수집
        futures = {
            executor.submit(tdc.fetch_tv_series_and_seasons, tv_id): tv_id
            for tv_id in id_list
        }

        for future in tqdm(as_completed(futures), total=len(futures), desc="TV 시리즈 + 시즌 수집"):
            try:
                series_data, seasons_list = future.result()

                if series_data:
                    series_results.append(series_data)

                if seasons_list:
                    seasons_results.extend(seasons_list)

            except Exception as e:
                tv_id = futures[future]
                print(f"오류 (TV ID: {tv_id}): {e}")

    print(f"\n시리즈 정보 수집 완료: {len(series_results):,}개")
    print(f"시즌 정보 수집 완료: {len(seasons_results):,}개")

    # ============================================================
    # 3단계: 저장
    # ============================================================

    # 시리즈 데이터 저장
    df_series = pd.DataFrame(series_results)
    df_series = df_series.sort_values(["first_air_date", "popularity"], ascending=[True, False])

    # 시즌 데이터 저장
    df_seasons = pd.DataFrame(seasons_results)
    df_seasons = df_seasons.sort_values(["series_id", "season_number"], ascending=[True, True])

    # CSV 저장
    df_series.to_csv(f"{OUTPUT_DIR}/tv_series_test.csv", index=False, encoding="utf-8-sig")
    df_seasons.to_csv(f"{OUTPUT_DIR}/tv_seasons_test.csv", index=False, encoding="utf-8-sig")

    elapsed = time.time() - total_start

    print("\n" + "=" * 90)
    print("========================== DONE ==========================")
    print(f"시리즈 데이터: {len(df_series):,}개")
    print(f"시즌 데이터: {len(df_seasons):,}개")
    print(f"소요시간: {elapsed / 60:.2f}분 ({elapsed:.2f}초)")
    print(f"저장 파일: tv_series_test.csv, tv_seasons_test.csv")
    print("=" * 90)


전체 기간: 2025-11-01 ~ 2025-11-10
분할 기간: 1개

[1/1] 기간: 2025-11-01 ~ 2025-11-10

📺 TV 시리즈 ID 수집: 2025-11-01 ~ 2025-11-10

[기간 1/1] 2025-11-01 ~ 2025-11-10
  [TV 시리즈] 총 384개 (20페이지) → 수집: 20페이지


  누적 ID: 384개

최종 수집: 384개 TV 시리즈 ID

해당 기간 내 수집된 ID: 384개 (누적 ID: 384개)

최종 ID 개수: 384개

상세 정보 멀티스레드 수집 시작...



TV 시리즈 + 시즌 수집: 100%|██████████| 384/384 [00:12<00:00, 29.82it/s]


시리즈 정보 수집 완료: 384개
시즌 정보 수집 완료: 388개

========================== DONE ==========================
시리즈 데이터: 384개
시즌 데이터: 388개
소요시간: 0.23분 (13.97초)
저장 파일: tv_series_test.csv, tv_seasons_test.csv


## IMDB 데이터 수집

In [3]:
import pandas as pd
import file_collector.imdb_collector as idc
import os

output_dir = f"{OUTPUT_DIR}/reviews"
os.makedirs(output_dir, exist_ok=True)

In [17]:
await idc.collect_imdb_reviews(
    input_csv_path=f"{OUTPUT_DIR}/movies_test.csv",   # 아까 만든 5035편 리스트
    output_csv_path=f"{OUTPUT_DIR}/reviews/movies_reviews_test.csv",   # 최종 리뷰 파일
    max_titles=None,               # 전체 5035편 다 돌리기
    max_reviews_per_title=None     # 영화당 리뷰 전부 수집
)

🚀 IMDb 리뷰 크롤링 시작
📂 입력: files/movies_test.csv
💾 출력: files/reviews/movies_reviews_test.csv
🎯 이번에 수집할 영화 수: 477
➖ Maria Vitória (tt33053905) 리뷰 없음
✅ Royal Runaways (tt10522278)  8/8개
➖ Hate Thyself (tt30526409) 리뷰 없음
➖ Race/America (tt36386809) 리뷰 없음
➖ Sottopelle (tt8962670) 리뷰 없음
➖ Cold White (tt33354418) 리뷰 없음
➖ Bimba (tt38624836) 리뷰 없음
➖ Watch the Skies (tt6267164) 리뷰 없음
➖ Os Perebas (tt10750578) 리뷰 없음
➖ Jastimari (tt35295423) 리뷰 없음
💾 배치 저장: 8개 리뷰
📊 진행 상황: 10/477 (경과 0.0분)
✅ Christmas in the Ballroom (tt37991625)  2/2개
✅ Christmas on Duty (tt37133902)  16/16개
✅ Patty Is Such a Girly Name (tt37530747)  1/1개
➖ Mega Trump (tt36890106) 리뷰 없음
➖ Bystander (tt31796702) 리뷰 없음
➖ Além do Horizonte - A Travessia (tt37540645) 리뷰 없음
✅ The Christmas Bookshop (tt35326037)  7/7개
➖ The Garden (tt38999706) 리뷰 없음
➖ Detox Inn (tt33098202) 리뷰 없음
➖ What If the Future Never Happened? (tt21060054) 리뷰 없음
💾 배치 저장: 26개 리뷰
📊 진행 상황: 20/477 (경과 0.1분)
➖ Noah's Daughter (tt38589225) 리뷰 없음
➖ Roubo do Cão (tt37458189) 

In [18]:
await idc.collect_imdb_ratings(
    input_csv_path=f"{OUTPUT_DIR}/movies_test.csv",   # 아까 만든 5035편 리스트
    output_csv_path=f"{OUTPUT_DIR}/movies_rating_test.csv",   # 최종 점수 파일
)

🚀 IMDB RATING COLLECTOR (2005-2015)

📂 데이터 로드 중...
⚠️  'first_air_date' 컬럼이 없습니다. 날짜 필터링을 건너뜁니다.
✅ 최종 필터링 (vote_count>=30 & imdb_id 존재): 12개

🚀 크롤링 시작
⚙️  Rate Limit: 2회/초
⏱️  예상 시간: 0분
📊 진행: 10/12 (83.3%) | 성공: 10 | 실패: 0 | 요청: 10회 | 속도: 163.6개/분 | ETA: 0분
📊 진행: 12/12 (100.0%) | 성공: 12 | 실패: 0 | 요청: 12회 | 속도: 135.8개/분 | ETA: 0분

💾 최종 저장 중...

🎉 크롤링 완료!
📌 시리즈: 12/12개 성공
📌 총 수집: 12개 (중복 제거 후)
📌 Rating 보유: 12개 (100.0%)
📌 평균 Rating: 6.21
📌 평균 Rating Count: 14578
📌 총 요청: 12회
⏱️  총 시간: 0.1분 (0.00시간)
📊 속도: 135.6개/분

📊 결과 샘플:
      imdb_id  imdb_rating  imdb_rating_count
0  tt31227572          7.3              63021
1  tt31425731          5.5              20829
2  tt29768334          7.5              36596
3  tt31193547          4.6                971
4  tt32398150          7.1                489
5  tt29567915          7.5              27195
6  tt32323252          6.2              11289
7  tt33293169          5.7               1780
8  tt38121182          6.5               3303
9  tt27604215  

In [4]:
await idc.collect_imdb_reviews(
    input_csv_path=f"{OUTPUT_DIR}/tv_series_test.csv",   # 아까 만든 5035편 리스트
    output_csv_path=f"{OUTPUT_DIR}/reviews/tv_reviews_test.csv",   # 최종 리뷰 파일
    max_titles=None,               # 전체 5035편 다 돌리기
    max_reviews_per_title=None     # 영화당 리뷰 전부 수집
)

🚀 IMDb 리뷰 크롤링 시작
📂 입력: files/tv_series_test.csv
💾 출력: files/reviews/tv_reviews_test.csv
📌 기존 리뷰 파일 발견 → 이미 수집된 imdb_id 스킵
   이미 수집된 영화 수: 57
🎯 이번에 수집할 영화 수: 54
➖ Dead End (tt32583026) 리뷰 없음
➖ Μία Νύχτα Μονο (tt38493618) 리뷰 없음
➖ Detective Naveed (tt32150299) 리뷰 없음
➖ L'appartamento - Sold Out (tt35663177) 리뷰 없음
➖ It's Time - E' Giunta L'ora (tt38111441) 리뷰 없음
➖ Het Zwaard van Damocles (tt38832337) 리뷰 없음
➖ A Happy Place (tt38961906) 리뷰 없음
➖ Lonely Hero (tt33365220) 리뷰 없음
➖ Holiday Bakeshop (tt38581726) 리뷰 없음
➖ Break from Apocalypse (tt38221468) 리뷰 없음
📊 진행 상황: 10/54 (경과 0.0분)
➖ Decalcomania (tt32880015) 리뷰 없음
➖ Hirayasumi (tt37810816) 리뷰 없음
➖ Wild Bees (tt38592526) 리뷰 없음
➖ Jagten på Danmarkshistorien (tt39151074) 리뷰 없음
➖ FAME/LESS (tt15783240) 리뷰 없음
➖ MDRJago: Bendies Rising (tt38626697) 리뷰 없음
➖ Mondes sauvages (tt38839652) 리뷰 없음
➖ Runaway (tt37342224) 리뷰 없음
➖ Soul Sucker (tt35669589) 리뷰 없음
➖ Sandi Toksvig's Hidden Wonders (tt38813959) 리뷰 없음
📊 진행 상황: 20/54 (경과 0.1분)
➖ Mälzers Meisterklasse

In [4]:
await idc.collect_imdb_ratings(
    input_csv_path=f"{OUTPUT_DIR}/tv_series_test.csv",
    output_csv_path=f"{OUTPUT_DIR}/tv_series_rating_test.csv",
)

🚀 IMDB RATING COLLECTOR (2005-2015)

📂 데이터 로드 중...
📅 날짜 필터링: 2005-01-01 ~ 2025-12-31
   원본: 384개 → 필터링 후: 384개
✅ 최종 필터링 (vote_count>=30 & imdb_id 존재): 10개

🚀 크롤링 시작
⚙️  Rate Limit: 2회/초
⏱️  예상 시간: 0분
📊 진행: 10/10 (100.0%) | 성공: 10 | 실패: 0 | 요청: 10회 | 속도: 142.9개/분 | ETA: 0분

💾 최종 저장 중...

🎉 크롤링 완료!
📌 시리즈: 10/10개 성공
📌 총 수집: 10개 (중복 제거 후)
📌 Rating 보유: 10개 (100.0%)
📌 평균 Rating: 7.22
📌 평균 Rating Count: 13338
📌 총 요청: 10회
⏱️  총 시간: 0.1분 (0.00시간)
📊 속도: 142.5개/분

📊 결과 샘플:
      imdb_id  imdb_rating  imdb_rating_count
0  tt33484460          6.5               3917
1  tt30291387          3.4              10058
2  tt36188448          7.8               3203
3  tt32253092          7.8               3554
4  tt22202452          8.1              72714
5  tt37890201          7.8               1950
6  tt31121364          7.7              12645
7  tt31314751          7.5              20735
8  tt33550045          7.5               2667
9  tt38491451          8.1               1940

✅ 결과 파일: files/tv_series_r

# 전처리

## 파일 전처리

In [4]:
import preprocessing.text as pt

movie_data = pd.read_csv(f"{OUTPUT_DIR}/movies_test.csv")
movie_rating = pd.read_csv(f"{OUTPUT_DIR}/movies_rating_test.csv")

movie_data = movie_data.merge(movie_rating, on="imdb_id", how="left")

drama_data = pd.read_csv(f"{OUTPUT_DIR}/tv_series_test.csv")
drama_rating = pd.read_csv(f"{OUTPUT_DIR}/tv_series_rating_test.csv")

drama_data = drama_data.merge(movie_rating, on="imdb_id", how="left")

In [9]:
# 사용법
movie_prep = pt.DataPreprocessor(type='movie')
movie_prep.preprocess(movie_data)
movie_prep.save(f'{OUTPUT_DIR}/movie')

drama_prep = pt.DataPreprocessor(type='tv')
drama_prep.preprocess(drama_data)
drama_prep.save(f'{OUTPUT_DIR}/drama')

  ✓ files/movie/00_movie_main.parquet
  ✓ files/movie/01_movie_roi.parquet
  ✓ files/movie/02_movie_production_countries.parquet
  ✓ files/movie/03_movie_genres.parquet
  ✓ files/movie/04_movie_providers.parquet
  ✓ files/drama/00_tv_main.parquet
  ✓ files/drama/02_tv_origin_countries.parquet
  ✓ files/drama/03_tv_genres.parquet
  ✓ files/drama/04_tv_providers.parquet


In [5]:
movie_review = pd.read_csv(f"{OUTPUT_DIR}/reviews/movies_reviews_test.csv")
drama_review = pd.read_csv(f"{OUTPUT_DIR}/reviews/tv_reviews_test.csv")
review = pd.concat([movie_review, drama_review])

In [6]:
movie_prep = pt.ReviewPreprocessor(english_threshold=0.9)
clean_reviews = movie_prep.preprocess(review)
movie_prep.save(f'{OUTPUT_DIR}/reviews/reviews_test.parquet')

▶ 텍스트 노이즈 제거 시작: 6,457개 행
  - 제거된 행: 44개
  - 남은 행: 6,413개






## 흥행 점수 계산

In [2]:
import preprocessing.scoring as sc
import preprocessing.sentiment as sp

In [5]:
movie_main = pd.read_parquet(f"{OUTPUT_DIR}/movie/00_movie_main.parquet")
tv_main = pd.read_parquet(f"{OUTPUT_DIR}/drama/00_tv_main.parquet")
main = pd.concat([
    movie_main[['imdb_id', 'tmdb_rating', 'tmdb_rating_count', 'imdb_rating', 'imdb_rating_count']],
    tv_main[['imdb_id', 'tmdb_rating', 'tmdb_rating_count', 'imdb_rating', 'imdb_rating_count']]
])
review = pd.read_parquet(f"{OUTPUT_DIR}/reviews/reviews_test.parquet")

In [5]:
analyzer = sp.SentimentAnalyzer()
analyzer.analyze(review, f"{OUTPUT_DIR}/reviews", text_column="review_text")

▶ Total rows: 6,413
▶ Total chunks: 1
▶ Existing chunks: 0 (will skip)
[2025-12-28 20:37:02] === SENTIMENT ANALYSIS START ===
[2025-12-28 20:37:02] CHUNK 0/0 - rows=6,413, range=[0:6,413]


[2025-12-28 20:49:52] CHUNK 00000 - DONE(12.83 min, 8.3 rows/sec) -> files/reviews/review_chunk_00000.parquet
[2025-12-28 20:49:52] === SENTIMENT ANALYSIS COMPLETED ===

✅ All chunks processed successfully!


In [6]:
review = pd.read_parquet(f"{OUTPUT_DIR}/reviews/review_chunk_00000.parquet").merge(pd.read_parquet(f"{OUTPUT_DIR}/reviews/reviews_test.parquet")[['helpful_ratio', 'review_id']], on='review_id', how='left')

In [10]:
sentiment_score = sc.compute_weighted_sentiment_score(review)
main_score = sc.calculate_ratings(main)

In [20]:
sc.calculate_score(main_score.merge(sentiment_score, on='imdb_id', how='inner'))

,imdb_id,hit_score
0,tt31227572,40.101378
1,tt31425731,31.025830
2,tt29768334,44.858438
3,tt31193547,16.817422
4,tt29567915,39.010789
5,tt32323252,35.887938
6,tt33293169,20.300631
7,tt38121182,18.700524
8,tt27604215,35.679517
9,tt35600079,31.486951
